In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk 
import string
import re
import codecs
import json
#%matplotlib inline
#pd.set_option('display.max_colwidth', 100)

import hdbscan
import nltk
import numpy as np
import pandas as pd
import random
import re
import spacy
import textacy
import csv
import sklearn

from gmplot import gmplot
from mapsplotlib import mapsplot as mplt
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag_sents
from pymprog import *
from sklearn.cluster import DBSCAN
from sklearn.neighbors import DistanceMetric
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd

import datetime
from time import strptime

from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
from sklearn.cluster import KMeans
import clustering_models as cl
import timestamp_graphs as tg
import warnings
warnings.filterwarnings('ignore')
from geograpy import extraction
from geograpy import places
import collections
from geograpy import places
from urllib.request import urlopen
import data_preprocessing_methods as dpm

In [2]:
colnames=['tweets', 'created_at', 'id','rt','lang','location','coordinates','Event'] 
df = pd.read_csv('relevant_with_coordinates.csv', names=colnames, header=None)
print(df.shape[0])
df.head(1)

106017


,tweets,created_at,id,rt,lang,location,coordinates,Event
0,News: Chinese author Mo Yan wins Nobel http://...,Thu Oct 11 11:08:32 +0000 2012,256350567369175040,False,en,Eindhoven,"{""lat"": 51.47601928708412, ""lon"": 5.4647688255...",0


In [3]:
df = df.drop(columns=['location', 'coordinates'])

In [4]:
import data_preprocessing_methods as dpm
temp = np.vectorize(dpm.remove_patterns)(df['tweets'],"@[\w]*","#[\w]*","RT")
df["text"] = np.array([text["input_text"] for text in temp])
df["hashtag"] = np.array([dpm.list_lower(text["hashtags"]) for text in temp])
df["user_mentions"] = np.array([text["user_mentions"] for text in temp]) 
df["RT"] = np.array([text["is_RT"] for text in temp])
df["URL"] = np.array([text["is_URL"] for text in temp])
df['Length_URL'] = np.array([len(temp) for temp in df['URL']])

In [10]:
temp = df.loc[df['Length_URL']!=0]
temp.shape[0]

31361

In [ ]:
def getPlaces(list_url):
    list_places = []
    for temp_url in list_url:
        try:
            ret = urlopen(temp_url)
            e = extraction.Extractor(url=ret.url)
            e.find_entities()
            name_places = e.places
            counter = collections.Counter(name_places)
            crucial_words = counter.most_common(3)
            for i in range(len(crucial_words)):
                list_places.append(crucial_words[i][0])
#             print(temp_url)
        except:
            pass
#     print(list_places)
    return list_places

In [ ]:
# [getPlaces(temp) for temp in df['URL']]

In [ ]:
# df['Places'] = np.array([getPlaces(temp) for temp in df['URL']])

In [ ]:
# df.to_csv(r'relevant_with_place.csv',index=False)

In [ ]:
import geograpy
# url = 'http://t.co/9QFHaJyE'
# places = geograpy.get_place_context(url=url)
from geograpy import extraction
from urllib.request import urlopen
try:
    ret = urlopen('https://paper.li/tppowers/Advertising#/')
    e = extraction.Extractor(url=ret.url)
    e.find_entities()

    # You can now access all of the places found by the Extractor
    print(e.places)
    from geograpy import places
    pc = places.PlaceContext(e.places)
    pc.set_countries()
    print(pc.countries) #['United States']
    pc.country_mentions
except:
    pass
    

In [11]:
colnames=['tweets', 'created_at', 'id','rt','lang','location','coordinates'] 
df_irrelevant = pd.read_csv('irrelevant_with_coordinates.csv', names=colnames, header=None)
print(df_irrelevant.shape[0])
df_irrelevant.head(1)

413986


,tweets,created_at,id,rt,lang,location,coordinates
0,I love ms.sivney class (Sports &amp; Entertaim...,Wed Oct 31 16:48:39 +0000 2012,263683917889040400,False,en,Macedonia,"{'lat': 41.82038761113906, 'lon': 22.008566323..."


In [12]:
temp = np.vectorize(dpm.remove_patterns)(df_irrelevant['tweets'],"@[\w]*","#[\w]*","RT")
df_irrelevant["text"] = np.array([text["input_text"] for text in temp])
df_irrelevant["hashtag"] = np.array([dpm.list_lower(text["hashtags"]) for text in temp])
df_irrelevant["RT"] = np.array([text["is_RT"] for text in temp])
df_irrelevant["URL"] = np.array([text["is_URL"] for text in temp])
df_irrelevant['Length_URL'] = np.array([len(temp) for temp in df_irrelevant['URL']])

In [ ]:
df_irrelevant['Places'] = np.array([getPlaces(temp) for temp in df['URL']])
df.to_csv(r'irrelevant_with_place.csv',index=False)